In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import requests
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

In [3]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str)-> str:
    """This function fetches the currency conversion factor from given base currency to target currency"""
    url = f"https://v6.exchangerate-api.com/v6/a411f1bec9afb8ef0ef05dd6/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()

In [4]:
@tool
def convert(base_currency_value: int, conversion_rate: float)-> float:
    """Given a currency conversion rate this function calculates the target currency value from a base currency value"""
    return base_currency_value * conversion_rate

In [5]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1747440002,
 'time_last_update_utc': 'Sat, 17 May 2025 00:00:02 +0000',
 'time_next_update_unix': 1747526402,
 'time_next_update_utc': 'Sun, 18 May 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.6035}

In [6]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
)

# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0.7,
# )

In [8]:
llm_tool = llm.bind_tools([get_conversion_factor, convert])
llm_tool

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001B124BB3C10>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor from given base currency to target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'Given a currency conversion rate this function calculates the target currency value from a base currency value', 'parameters': {'properties': {'base_currency_value': {'type': 'integer'}, 'conversion_rate': {'type': 'number'}}, 'required': ['base_currency_value', 'conversion_rate

In [9]:
llm_tool.invoke('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--a662a5c9-341c-4d0b-8024-96f0eacbc2bd-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': 'abecfe70-765e-4295-889a-c85f7c1bcda9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 96, 'output_tokens': 13, 'total_tokens': 109, 'input_token_details': {'cache_read': 0}})

In [10]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\Adwait\AppData\Local\Temp\ipykernel_26544\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [11]:
# --- Step 6: Run the Agent ---
user_query = "What is the conversion factor from USD to INR, and based on that can you convert 10 USD to INR"

response = agent_executor.invoke({"input": user_query})
response



> Entering new AgentExecutor chain...
Thought:I need to get the USD to INR conversion factor first using the get_conversion_factor tool.

Action:
```json
{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "USD",
    "target_currency": "INR"
  }
}
```
Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1747440002, 'time_last_update_utc': 'Sat, 17 May 2025 00:00:02 +0000', 'time_next_update_unix': 1747526402, 'time_next_update_utc': 'Sun, 18 May 2025 00:00:02 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 85.6035}
Thought:I have the conversion rate. Now I can use the convert tool to convert 10 USD to INR.

Action:
```json
{
  "action": "convert",
  "action_input": {
    "base_currency_value": 10,
    "conversion_rate": 85.6035
  }
}
```
Observation: 856.035
Thought:I know what to respond.

Action:
```json
{
  "act

{'input': 'What is the conversion factor from USD to INR, and based on that can you convert 10 USD to INR',
 'output': 'The conversion factor from USD to INR is 85.6035.  10 USD is equal to 856.035 INR.'}